In [30]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#Hidden 
from PIL import Image
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout

In [31]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [32]:
print(f'the shape of X_train is :{X_train.shape}')
print(f'the shape of y_train is :{y_train.shape}')


the shape of X_train is :(10, 3)
the shape of y_train is :(10,)


In [33]:
# compute entropy
def compute_entropy(y):
    if len(y)==0:
        return 0
    p_1 =sum(y)/len(y)
    if p_1==0 or p_1==1:
        return 0
    entropy = -p_1*np.log2(p_1)-(1-p_1)*np.log2(1-p_1)
    return entropy

In [34]:
# Compute entropy at the root node (i.e with all the examples)

print(f'entropy at the root node:{compute_entropy(y_train)}')


entropy at the root node:1.0


In [35]:
# split the data
def split_data(X,node_indices,feature):
    left_indices = []
    right_indices = []
    for i in node_indices:
        if X[i][feature]== 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices,right_indices


In [39]:
def generate_node_image(node_indices):
    image_paths = ["%d.png" % idx for idx in node_indices]
    images = [Image.open(x) for x in image_paths]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    
    new_im = new_im.resize((int(total_width*len(node_indices)/10), int(max_height*len(node_indices)/10)))
    
    return new_im

In [42]:
def generate_split_viz(node_indices, left_indices, right_indices, feature):
    
    G=nx.DiGraph()
    
    indices_list = [node_indices, left_indices, right_indices]
    for idx, indices in enumerate(indices_list):
        G.add_node(idx,image= generate_node_image(indices))

    G.add_edge(0,1)
    G.add_edge(0,2)

    pos = graphviz_layout(G, prog="dot")

    fig=plt.figure()
    ax=plt.subplot(111)
    ax.set_aspect('equal')
    nx.draw_networkx_edges(G,pos,ax=ax, arrows=True, arrowsize=40)
    
    trans=ax.transData.transform
    trans2=fig.transFigure.inverted().transform

    feature_name = ["Brown Cap", "Tapering Stalk Shape", "Solitary"][feature]
    ax_name = ["Splitting on %s" % feature_name , "Left: %s = 1" % feature_name, "Right: %s = 0" % feature_name]
    for idx, n in enumerate(G):
        xx,yy=trans(pos[n]) # figure coordinates
        xa,ya=trans2((xx,yy)) # axes coordinates
        piesize = len(indices_list[idx])/9
        p2=piesize/2.0
        a = plt.axes([xa-p2,ya-p2, piesize, piesize])
        a.set_aspect('equal')
        a.imshow(G.nodes[n]['image'])
        a.axis('off')
        a.set_title(ax_name[idx])
    ax.axis('off')
    plt.show()

In [43]:
# Case 1
root_indices = [0,1,2,3,4,5,6,7,8,9]
feature = 0
left_indices,right_indices = split_data(X_train,root_indices,feature)

print('Case 1:')
print('Left indices:',left_indices)
print('Right indices',right_indices)

#Case2
root_indices_subset = [0,2,4,6,8]
left_indices,right_indices = split_data(X_train,root_indices_subset,feature)

print('Case2:')
print('Left indices:',left_indices)
print('Right indices:',right_indices)
# Visualize the split

#generate_split_viz(root_indices_subset,left_indices,right_indices,feature) giving error



Case 1:
Left indices: [0, 1, 2, 3, 4, 7, 9]
Right indices [5, 6, 8]
Case2:
Left indices: [0, 2, 4]
Right indices: [6, 8]


In [46]:
# Compute information gain
def compute_information_gain(X,y,node_indices,feature):
    left_indices,right_indices = split_data(X,node_indices,feature)
    
    #some useful variables
    X_node,y_node = X[node_indices],y[node_indices]
    X_left,y_left = X[left_indices],y[left_indices]
    X_right,y_right = X[right_indices],y[right_indices]
    
    information_gain= 0
    node_entropy = compute_entropy(y_node)
    w_left = len(X_left)/len(y_node)
    w_right= len(X_right)/len(y_node)
    left_entropy=compute_entropy(y_left)
    right_entropy= compute_entropy(y_right)
    
    gain = w_left*left_entropy + w_right*right_entropy
    
    information_gain = node_entropy-gain
    return information_gain

In [47]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)



Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377


In [50]:
# get best split

def get_best_split(X,y,node_indices):
    num_features=X.shape[1]
    best_feature = -1
    
    m=0
    for i in range(num_features):
        info_gain = compute_information_gain(X,y,node_indices,i)
        if m<info_gain:
            m = info_gain
            best_feature =i
    return best_feature

In [51]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)



Best feature to split on: 2


In [53]:
tree = []

def build_tree_recuresive(X,y,node_indices,branch_name,max_depth,current_depth):
     # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)